In [ ]:
# 43679 -- Interactive Visualization
# 2025 - 2026
# 2nd semester
# Lab 1 - EDA (guided)
# ver 1.2
# 24022026 - Cosmetics; added rationale for task in scope of course

# Lab 02<br>Task 2: Guided EDA and Data Cleaning

The purpose of this task you to introduce you to the basic steps of performing data preparation for a dataset with several illustrative quality issues. In most situations you already have the basic code to be run; in others, you need to infer from existing code to complete the step. What is important here is for you to be able to identify the issues, understand the tools and approaches that may help tackling them, and acquire a systematic way of thinking about data preparation. **This is something you will need to do for your final project**.

**Don't just run the code. Understand why it is needed and what it is doing**

**NOTE**: For those cells asking questions or with tables that can be filled, you can just double-click the cell and edit it with your answers and rationale

**Dataset:** `dataset_A_indie_game_telemetry.csv`

---

### Objectives

By the end of this task you will be able to:
- Use **SweetViz** to rapidly profile a dataset and identify issues
- Use **D-Tale** to navigate and inspect a dataframe interactively
- Use **pandas** to fix the most common categories of data quality problems
- Make and justify cleaning decisions rather than applying fixes mechanically

### Tools and their roles in this task

| Tool | Role |
|---|---|
| **SweetViz** | Automated profiling: generate a report, triage what needs fixing |
| **D-Tale** | Interactive navigation: browse rows, inspect value counts, confirm fixes visually |
| **pandas** | All actual cleaning: every transformation is explicit, reproducible code |

---

## Part 1 — Setup and First Look

In [ ]:
import pandas as pd
import sweetviz as sv
import dtale
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the raw dataset — do NOT clean anything yet
df = pd.read_csv('dataset_A_indie_game_telemetry.csv')

print(f'Shape: {df.shape}')
df.head()

In [ ]:
# Column names and types as pandas inferred them
print(df.dtypes)

> **⚠️ Notice:** Several columns that should be boolean (`crash_flag`, `is_featured_event`, `is_long_session`) or
> numeric (`purchase_amount`) have been inferred as `object`. This is your first signal that something is wrong.

---

## Part 2: Automated Profiling with SweetViz

SweetViz generates a visual report for the entire dataset in one call. Think of it as a **triage tool** — it shows you *where* to look; the actual investigation and fixing happens afterwards.

In [ ]:
# Generate the profiling report (~30–60 seconds)
report = sv.analyze(df)
report.show_html('sweetviz_raw_report.html', open_browser=True)
print('Report saved. Open sweetviz_raw_report.html in your browser.')

Open the report and answer the following before moving on.

| Question | Your finding |
|---|---|
| Which columns have missing values? Which has the most? | *...* |
| Which columns are shown as TEXT but should be boolean or numeric? | *...* |
| Are there numeric columns with suspicious ranges? | *...* |
| How many distinct values does `region` have? Does that seem right? | *...* |
| What is unusual about `purchase_amount`? | *...* |



---

## Part 3: Navigate and Inspect with D-Tale

Before writing any cleaning code, use D-Tale to browse the raw data and *see* the problems with your own eyes. You will not clean anything here — D-Tale is your inspection tool.

**Launch D-Tale:**

In [ ]:
# This will open an interactive D-Tale session in your browser, allowing you to explore the raw dataset in more detail
# The subprocess=True argument allows D-Tale to run in a separate process, which can help avoid issues with Jupyter notebooks
# Otherwise, D-Tale would block the notebook until you close the D-Tale session, which is not ideal for interactive exploration
d = dtale.show(df, host='127.0.0.1', subprocess=True, open_browser=True)
print('=' * 50)
print('D-Tale is running.')
print('Open this URL in your browser:', d._url)
print('In VS Code: Ctrl+click the URL above.')
print('=' * 50)

### Inspection checklist

Use D-Tale to confirm each issue SweetViz flagged. For each column, click the column header → **Describe** to see value counts and distribution.

| What to inspect | How to do it in D-Tale | What you should see |
|---|---|---|
| `crash_flag` unique values | Column header → Describe | 8 variants of True/False |
| `region` unique values | Column header → Describe | ~32 variants of 5 region names |
| `input_method` unique values | Column header → Describe | A typo: `controllr` |
| `purchase_amount` raw values | Sort column ascending | Some values use comma: `1,80` |
| `avg_fps` distribution | Column header → Describe | Max of 10,000 — clearly wrong |
| Missing values overview | Top menu → Describe (all columns) | `gpu_model` dominates |

<br>

> Once you have seen the problems in the raw data, come back to the notebook for cleaning.

---

## Part 4: Clean with Pandas

We will work through seven issue categories. Each section follows the same pattern:
1. **Inspect** — confirm the problem in code
2. **Fix** — apply the pandas transformation
3. **Verify** — check the result

We work on a copy of the original dataframe so the raw data is always available for comparison.

In [ ]:
# Always work on a copy — keep df as the unchanged original
df_clean = df.copy()

---

### 4.1. Boolean columns: inconsistent encoding

Three columns (`crash_flag`, `is_featured_event`, `is_long_session`) each have **8 different representations** of the same two values: `True`, `False`, `true`, `false`, `1`, `0`, `Yes`, `No`.

The fix is to define an explicit mapping and apply it with `.map()`.

In [ ]:
# Inspect — confirm the problem
print('crash_flag unique values:', sorted(df_clean['crash_flag'].dropna().unique()))

In [ ]:
# Define the mapping for replacements
# Why did I place True:True and False: False? Ideas?

bool_map = {
    'True': True,  'true': True,  '1': True,  'Yes': True,  True: True,
    'False': False, 'false': False, '0': False, 'No': False, False: False
}

df_clean['crash_flag'] = df_clean['crash_flag'].map(bool_map)

print('crash_flag after mapping:')
print(df_clean['crash_flag'].value_counts())
print('Nulls:', df_clean['crash_flag'].isna().sum())

In [ ]:
# TO DO:
# Apply the same mapping to the other two boolean columns
# Follow the same pattern as above for is_featured_event and is_long_session


---

### 4.2. Categorical columns: case and whitespace inconsistency

Four columns have values that are logically identical but differ in case or surrounding whitespace:
- `region` — 32 variants of 5 values (e.g. `us-west`, `US-WEST`, `Us-west`, `' us-west '`)
- `map_name` — 36 variants of 6 values
- `platform` — 32 variants of 6 values
- `input_method` — 30 variants, including a **typo**: `controllr`

The fix uses pandas string methods: `.str.strip()` removes surrounding whitespace, `.str.lower()` normalises case. They can be chained.

In [ ]:
# Inspect — how many unique values before cleaning?
print('region unique before:', df_clean['region'].unique())

In [ ]:
# Fix region: strip whitespace and convert to lowercase
df_clean['region'] = df_clean['region'].str.strip().str.lower()

# Verify
print('region unique after:', df_clean['region'].unique())
print(df_clean['region'].value_counts())

In [ ]:
# TO DO: 
# Apply the same strip + lower to map_name and platform
# Follow the same pattern as above


In [ ]:
# input_method needs an extra step: fix the typo and standardise kb/m → kbm

# Step 0: Inspect
print('input_method unique before:', df_clean['input_method'].unique())

# Step 1: strip and lowercase first
df_clean['input_method'] = df_clean['input_method'].str.strip().str.lower()

# Step 2: fix the two inconsistencies with replace()
df_clean['input_method'] = df_clean['input_method'].replace({
    'controllr': 'controller',   
    'kb/m': 'kbm'                
})

# Verify — should now show exactly 3 unique values
print('input_method unique after:', df_clean['input_method'].unique())
print(df_clean['input_method'].value_counts())

---

### 4.3. `purchase_amount`: comma as decimal separator

About 12% of rows use a comma instead of a decimal point (`1,80` instead of `1.80`). This prevented pandas from reading the column as numeric, so it was loaded as `object`.

The fix: replace the comma in the string, then convert the column type.

In [ ]:
# Inspect — how many rows have a comma?
comma_rows = df_clean['purchase_amount'].astype(str).str.contains(',', na=False)
print(f'Rows with comma separator: {comma_rows.sum()}')
print('Examples:', df_clean.loc[comma_rows, 'purchase_amount'].unique()[:6])

In [ ]:
# Fix: replace comma with decimal point, then convert to float
df_clean['purchase_amount'] = (
    df_clean['purchase_amount']
    .astype(str)                            # ensure we are working with strings
    .str.replace(',', '.', regex=False)     # swap the separator
    .replace('nan', float('nan'))           # restore actual NaN rows
    .astype(float)                          # convert to numeric
)

# Verify
print('dtype:', df_clean['purchase_amount'].dtype)
print(df_clean['purchase_amount'].describe().round(2))

---

### 4.4. Missing values: decisions and strategy

Not all missing values are the same. Before deciding what to do, you need to understand *why* the value is missing — the reason determines the correct action.

| Column | Missing | Why | Decision |
|---|---|---|---|
| `gpu_model` | 66.7% | Console/mobile players have no GPU | Keep column — missingness is meaningful |
| `build_version` | 16.5% | Not logged in older sessions | Keep as NaN — valid historical absence |
| `device_temp_c` | 4.9% | Sensor not available on some devices | Keep as NaN |
| `session_length_s` | 1.0% | Session ended abnormally | Drop missing rows now; fix negatives/outliers after datetime correction (section 4.6) |
| `ping_ms`, `purchase_amount`, `end_time` | < 2% | Sporadic gaps | Keep as NaN |

<br>

> **⚠️ Context always matters.** There is no universal rule for missing values. The decisions above are reasonable for this dataset and analytical goal, but a different context might lead to different choices.


In [ ]:
# Inspect — missing value counts across all columns
missing = df_clean.isnull().sum()
missing_pct = (missing / len(df_clean) * 100).round(1)
pd.DataFrame({'missing': missing, '%': missing_pct})[missing > 0]

In [ ]:
# session_length_s: drop rows where it is missing
# Rationale: session duration is a core metric — a session with no recorded
# duration is structurally incomplete and cannot be used for most analyses.
# These 98 rows represent <1% of the dataset, so dropping is safe.

rows_before = len(df_clean)
df_clean = df_clean.dropna(subset=['session_length_s'])

print(f'Rows dropped: {rows_before - len(df_clean)}')
print(f'Rows remaining: {len(df_clean)}')

---

### 4.5. Outliers: `avg_fps`

The `avg_fps` column has a maximum of 10,000 fps — physically impossible for a game running in real time. The 75th percentile is ~82 fps, confirming that 10,000 is a logging error, not an extreme but plausible value.

**Decision:** set values above 300 fps to `NaN` rather than dropping the entire row. The rest of the data in those rows (crash flag, purchase amount, session type) is likely still valid — it would be wasteful to discard it.

In [ ]:
# Inspect — how many rows are affected?
threshold = 300
outlier_mask = df_clean['avg_fps'] > threshold
print(f'Rows with avg_fps > {threshold}: {outlier_mask.sum()}')
print('\navg_fps distribution (before fix):')
print(df_clean['avg_fps'].describe().round(1))

In [ ]:
# Fix: set outlier values to NaN using .loc with a boolean mask
df_clean.loc[outlier_mask, 'avg_fps'] = float('nan')

# Verify — max should now be well below 300
print('avg_fps distribution (after fix):')
print(df_clean['avg_fps'].describe().round(1))

---

### 4.6. Datetime columns: mixed formats

The `start_time` and `end_time` columns contain timestamps in at least four different formats:

```
2025-07-18T18:32:00Z          : ISO 8601 with UTC marker
2025-07-18 20:03:21-05:00     : ISO 8601 with UTC offset
20/10/2025 02:49              : European DD/MM/YYYY
08/01/2025 06:35              : Ambiguous: US MM/DD or European DD/MM?
```

Mixed datetime formats are one of the most complex cleaning problems because some ambiguities cannot be resolved automatically -- `08/01/2025` could be August 1st or January 8th, and no algorithm can determine which without external context.

> **Connection to `session_length_s`:** The negative values and extreme outliers we saw earlier in `session_length_s` are not independent errors -- they are a *consequence* of this datetime problem. When `start_time` and `end_time` were recorded in different formats and misinterpreted, the pre-computed duration came out wrong. After fixing the timestamps, we will recompute `session_length_s` from scratch and validate the result.


In [ ]:
# Inspect — what does start_time actually look like?
print('Sample values from start_time:')
print(df_clean['start_time'].dropna().sample(8, random_state=42).tolist())

In [ ]:
# Fix: pd.to_datetime with utc=True normalises all timezone-aware formats to UTC.
# errors='coerce' converts anything it cannot parse to NaT (Not a Time) instead of crashing.
df_clean['start_time'] = pd.to_datetime(df_clean['start_time'], utc=True, errors='coerce')
df_clean['end_time']   = pd.to_datetime(df_clean['end_time'],   utc=True, errors='coerce')

# Verify — check how many rows could not be parsed
print('start_time dtype:', df_clean['start_time'].dtype)
print('Unparsed start_time (NaT):', df_clean['start_time'].isna().sum())
print('Unparsed end_time (NaT):  ', df_clean['end_time'].isna().sum())

In [ ]:
# Recompute session_length_s from the corrected timestamps
# Now that start_time and end_time are both timezone-aware UTC datetimes,
# the subtraction is unambiguous. We convert the result to seconds.
df_clean['session_length_s'] = (
    df_clean['end_time'] - df_clean['start_time']
).dt.total_seconds()

print('session_length_s after recomputation:')
print(df_clean['session_length_s'].describe().round(1))
print(f'\nNegative values: {(df_clean["session_length_s"] < 0).sum()}')
print(f'> 8h (28800s):   {(df_clean["session_length_s"] > 28800).sum()}')


In [ ]:
# Any remaining negative values are rows where timestamps were genuinely
# ambiguous and could not be resolved -- the computed duration is meaningless.
# Set them to NaN rather than dropping the row.

neg_mask = df_clean['session_length_s'] < 0
df_clean.loc[neg_mask, 'session_length_s'] = float('nan')
print(f'Negative durations set to NaN: {neg_mask.sum()}')

# Values above 8 hours (28800s) are suspicious for a game session.
# Inspect them before deciding.

long_mask = df_clean['session_length_s'] > 28800
print(f'\nSessions > 8h: {long_mask.sum()}')
print(df_clean.loc[long_mask, ['session_length_s', 'start_time', 'end_time']].head(5).to_string())

# Decision: sessions > 8h are almost certainly logging errors (game left running,
# server not recording session end). Set to NaN.
# As always — this threshold is a judgement call that depends on the game and context.
df_clean.loc[long_mask, 'session_length_s'] = float('nan')
print(f'\nSessions > 8h set to NaN: {long_mask.sum()}')
print('\nFinal session_length_s distribution:')
print(df_clean['session_length_s'].describe().round(1))


> **Note:** The number of NaT values above reflects rows where pandas could not parse the format unambiguously. These are not errors in the code — they are genuinely ambiguous records that require a domain decision to resolve (e.g., knowing that the data source always uses DD/MM/YYYY).

---

** **OPTIONAL** — explore the unparsed rows**

If you want to go further, the cells below help you examine which formats failed and attempt a two-pass parsing strategy. This is optional and not required to complete the lab.



In [ ]:
# OPTIONAL — Step 1: inspect the unparsed rows
# We use the index of df_clean (not a boolean mask) to look up raw values in df,
# since the two dataframes have different lengths after the dropna() in step 4.4.
unparsed_idx = df_clean.index[df_clean['start_time'].isna()]
raw_start = df.loc[unparsed_idx, 'start_time'].dropna()

print(f'Rows still unparsed: {len(unparsed_idx)}')
print('\nSample raw values:')
print(raw_start.unique()[:12])


In [ ]:
# OPTIONAL: Step 2: define a systematic multi-format parser
#
# Rather than guessing with dayfirst=True, we try explicit format strings
# in sequence and stop as soon as one succeeds for each row.
# This is precise and transparent — no silent inference.

def try_formats(series, formats):
    """Try explicit datetime format strings in order.
    Returns a UTC-aware Series; rows that match no format remain NaT."""
    result = pd.Series(pd.NaT, index=series.index, dtype='datetime64[ns, UTC]')
    remaining = series.copy()
    for fmt in formats:
        parsed = pd.to_datetime(remaining, format=fmt, errors='coerce', utc=True)
        resolved_idx = parsed.index[parsed.notna()]   # use index labels, not boolean mask
        result.loc[resolved_idx] = parsed.loc[resolved_idx]
        remaining = remaining.drop(index=resolved_idx) # drop resolved rows by label
    return result

# Format strings to try, in order of specificity
# DD/MM/YYYY is tried before MM/DD/YYYY because values where day > 12
# can only be DD/MM — those are unambiguous and should be resolved first.
# Values where day <= 12 will match both formats; the first one wins.
# Those cases are genuinely ambiguous — we flag them separately below.
candidate_formats = [
    '%d/%m/%Y %H:%M',   # European with time: 20/10/2025 14:30
    '%m/%d/%Y %H:%M',   # US with time:       10/20/2025 14:30
    '%d/%m/%Y',         # European date only: 20/10/2025
    '%m/%d/%Y',         # US date only:       10/20/2025
]


In [ ]:
# OPTIONAL: Step 3: apply the systematic parser to unparsed rows
raw_start = df.loc[unparsed_idx, 'start_time']
raw_end   = df.loc[unparsed_idx, 'end_time']

resolved_start = try_formats(raw_start, candidate_formats)
resolved_end   = try_formats(raw_end,   candidate_formats)

df_clean.loc[unparsed_idx, 'start_time'] = resolved_start
df_clean.loc[unparsed_idx, 'end_time']   = resolved_end

print(f'Resolved in second pass: {resolved_start.notna().sum()}')
print(f'Still NaT (truly ambiguous): {df_clean["start_time"].isna().sum()}')


In [ ]:
# OPTIONAL: Step 4: inspect truly ambiguous rows
# These are rows where day <= 12, making both DD/MM and MM/DD valid.
# No algorithm can resolve them without knowing the data source convention.
# They remain NaT — do not silently guess.
still_nat_idx = df_clean.index[df_clean['start_time'].isna()]
if len(still_nat_idx) > 0:
    print('Truly ambiguous timestamps (cannot resolve without domain knowledge):')
    print(df.loc[still_nat_idx, ['start_time', 'end_time']].head(10).to_string())
else:
    print('All timestamps resolved.')


In [ ]:
# OPTIONAL: Step 5: recompute session_length_s with the newly resolved timestamps
# More rows now have valid start_time and end_time, so more durations can be recovered.
df_clean['session_length_s'] = (
    df_clean['end_time'] - df_clean['start_time']
).dt.total_seconds()

# Re-apply the same validation as before
neg_mask  = df_clean['session_length_s'] < 0
long_mask = df_clean['session_length_s'] > 28800
df_clean.loc[neg_mask | long_mask, 'session_length_s'] = float('nan')

print('session_length_s after second-pass recomputation:')
print(df_clean['session_length_s'].describe().round(1))
print(f'\nNaN values: {df_clean["session_length_s"].isna().sum()}')


</details>

---

## Part 5: Verify with D-Tale

Reload the cleaned dataframe into D-Tale and visually confirm the fixes. This is a quick sanity check — you are looking for anything that looks wrong before committing to the cleaned dataset.

In [ ]:
# Shut down the previous D-Tale instance and reload with the clean data
d.kill()
d_clean = dtale.show(df_clean, host='127.0.0.1', subprocess=True, open_browser=True)
print('Open cleaned data in D-Tale:', d_clean._url)

In D-Tale, verify the following:

| Column | What to check | Expected result |
|---|---|---|
| `crash_flag` | Describe → value counts | Only `True` and `False` |
| `region` | Describe → value counts | Exactly 5 values, all lowercase |
| `input_method` | Describe → value counts | Exactly 3 values, no `controllr` |
| `purchase_amount` | Describe → dtype and range | float64, no commas |
| `avg_fps` | Describe → max | Below 300 |
| `session_length_s` | Describe → min and max | No negatives, no values > 28800 |
| `start_time` | Describe → dtype | datetime64 |

## Part 6: Compare initial and clean datasets with SweetViz

In [ ]:
# Debug code; sometimes, sweetviz is not able to compare columns due to data type changes that are incompatible
# This code just goes around column by column to identify any column that gives an error. Otherwise, SweetViz
# just crashes without any major explanation

# Test comparison column by column
# for col in df_clean.columns:
#     try:
#         sv.compare([df[[col]], 'Raw'], [df_clean[[col]].reset_index(drop=True), 'Cleaned'])
#     except Exception as e:
#         print(f"FAIL: {col} — {e}")
#     else:
#         print(f"ok:   {col}")

In [ ]:
# Compare both versions of the dataset using SweetViz... 
# Not perfect, but some basic information (e.g., works bad with booleans vs categorical in crash_flag)
# needed to exclude these two because we converted them to datetime and sweetviz is not able to compare it with the original data types

exclude = ['start_time', 'end_time'] 

compare = sv.compare(
    [df.drop(columns=exclude), 'Raw'],
    [df_clean.drop(columns=exclude).reset_index(drop=True), 'Cleaned']
)
compare.show_html('sweetviz_comparison_report.html', open_browser=True)
print('Comparison report saved.')

In the comparison report, check that:
- Boolean columns changed from TEXT → BOOL with only 2 distinct values
- Categorical columns show dramatically reduced DISTINCT counts
- `purchase_amount` changed from TEXT → NUMERIC
- `avg_fps` maximum is no longer 10,000
- `session_length_s` shows 0 missing

---

## Part 7: Save the Cleaned Dataset

In [ ]:
df_clean.to_csv('dataset_A_indie_game_telemetry_clean.csv', index=False)
print(f'Saved: {len(df_clean)} rows, {len(df_clean.columns)} columns')

---

## Key Takeaways

**Three tools, three roles — they complement each other:**
- **SweetViz** surfaces issues fast but cannot fix them:  use it for triage and validation
- **D-Tale** lets you see the data as a human would:  use it to understand problems before and after fixing them
- **pandas** is where all actual cleaning happens: explicit, reproducible, and version-controllable

**Cleaning decisions are not mechanical:**
- Dropping `session_length_s` nulls was justified here: it would not be in every context
- Setting `avg_fps` outliers to NaN (not dropping rows) preserved valid data in other columns
- `gpu_model` missingness is structurally meaningful: imputing it would destroy information

**Common issue categories you have now fixed with pandas:**

| Issue | pandas approach |
|---|---|
| Boolean encoding chaos | `.map(bool_map)` |
| Case / whitespace inconsistency | `.str.strip().str.lower()` |
| Typos in categories | `.replace({'controllr': 'controller'})` |
| Wrong decimal separator | `.str.replace(',', '.')` + `.astype(float)` |
| Structural missing values | `dropna(subset=[...])` with explicit rationale |
| Outliers | Boolean mask + `.loc[mask, col] = NaN` |
| Mixed datetime formats | `pd.to_datetime(utc=True, errors='coerce')` |
